Lab03
=======

### Context
#### Class Imbalanced
+ AutoEncoder
+ Sampling

## AutoEncoder
AutoEncoder는 입력과 똑같은 출력을 만들어 내도록 하는 Neural Network의 일종입니다.<br>
데이터를 압축하여 Latent 벡터를 생성하는 Encoder 부분과 Latent 벡터를 가지고 기존의 데이터를 복원하는 Decoder 로 구성되어 있습니다.<br>
여기서 한가지 가정은 AutoEncoder가 생성하는 Latent 벡터는 입력 데이터를 아주 잘 표현하는 벡터이며, 해당 벡터를 복원했을 때 기존의 데이터가 출력 된다는 점 입니다.<br>
이러한 가정으로인해 우리가 AutoEncoder를 차원 축소 기법으로 사용할 수 있습니다. 

![AudoEncoder](./Images/AutoEncoder.png)

### 이상치 탐지에서의 AutoEncoder
위에서 이야기한 AutoEncoder가 생성하는 Latent 벡터는 정상 데이터를 잘 표현하는 벡터입니다. <br>
여기에서 만약 입력으로 정상적인 데이터가 아닌 불량한 데이터를 입력으로 넣는다면 어떻게 될까요?<br>
AutoEncoder를 정상 데이터로 학습시켜 네트워크를 구성하고, 해당 네트워크에 불량 데이터를 넣는다면 복원되는 데이터는 정상적으로 복원되지 않게 됩니다.<br>
이러한 방식으로 우리는 AutoEncoder를 이상치 탐지에도 사용할 수 있습니다.<br>

### 클래스 불균형 문제에서의 AutoEncoder
정상적인 데이터로 잘 학습된 AutoEncoder는 이상치를 잘 구분하는 모델이 됩니다.<br>
이를 통해 우리는 데이터에서 정상적이지 않은 데이터를 걸러내어 데이터의 수를 줄이는(UnderSample) 기법으로 사용할 수 있습니다.<br>

### 우리가 만들어볼 AutoEncoder는?
AutoEncoder에서 높은 차원의 입력이 낮은 차원의 각 층을 통과할 때 마다 입력 데이터는 정보를 손실하게 됩니다.<br>
여기에서 낮은 차원으로 내려갈 때 차원이 급격하게 감소하면 정보의 손실은 더욱 심해지게 됩니다.<br>
그래서 이번 실습에서는 입력 데이터의 정보를 덜 손실하면서 잘 복원하는 Stacked AutoEncoder 모델을 만들어 보도록 하겠습니다.<br>

In [ ]:
from os.path import join

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras import layers, models, optimizers, Input
from keras.utils import to_categorical

import matplotlib.pyplot as plt

datapath = join('data','fashion-mnist.npy')
labelpath = join('data','fashion-Label.npy')

# 케라스 백 엔드인 텐서플로우의 세션 설정을 불러옵니다.
from keras.backend.tensorflow_backend import set_session

# 텐서플로우의 ConfigProto() 객체에 학습시 적용할 옵션들을 명시적으로 설정할 수 있습니다.
config = tf.ConfigProto()
# GPU 옵션으로 allow_grouth를 True로 설정합니다.
config.gpu_options.allow_growth = True 

# 텐서플로우는 세션이라는 실행 단위를 가지고 있는데, 해당 세션에 적용할 옵션을 담고있는 Config객체를 전달합니다.
sess = tf.Session(config=config)
# 설정한 세션을 현재 프로세스에 적용합니다.
set_session(sess)  

## 1. 데이터 살펴보기
이번에 사용할 데이터는 손글씨 데이터로 유명한 MNIST 데이터의 옷 버전인 fashion-MNIST 입니다.<br>
실습의 편의를 위해 데이터는 28*28 행렬로 변환되어 저장된 상태입니다.<br>
data 디렉토리에 준비된 데이터와 라벨을 가져옵니다.<br>


#### 레이블 설명
0. 티셔츠/탑 
1. 바지 
2. 풀오버(스웨터의 일종) 
3. 드레스 
4. 코트 
5. 샌들 
6. 셔츠 
7. 스니커즈 
8. 가방 
9. 앵클 부츠

In [ ]:
X = np.load(datapath)
labels = np.load(labelpath)

In [ ]:
img=plt.imshow(X[0].reshape((28,28)))

## 2. 간단한 전처리
fashion-MNIST 데이터는 28*28에 픽셀마다 0~255 범위의 값을 가진 행렬입니다. <br>
각 픽셀에 대해 0~1 사이의 값을 갖도록 255로 나누어줍니다.<br>
그리고 네트워크에 데이터를 입력하기 편하게 하도록 2차원 이미지를 벡터형태로 펼치겠습니다.<br>
numpy.reshape() 함수를 통해 행렬의 차원을 변경할 수 있습니다.

In [ ]:
X = X/255
X = X.reshape(len(X),784)

값의 범위가 0과 1사이로 잘 되었는지 확인해봅시다.<br>
numpy.max(), numpy.min() 으로 확인해 볼 수 있습니다.

In [ ]:
print('최대 : {}, 최소 : {}'.format(np.max(X), np.min(X)))

라벨을 쉽게 확인할 수 있도록 파이썬 딕셔너리로 정리하겠습니다.

In [ ]:
label_dict = {0:'티셔츠/탑', 1:'바지', 2:'풀오버', 3:'드레스', 4:'코트', 5:'샌들',
             6:'셔츠', 7:'스니커즈', 8:'가방', 9:'앵클 부츠'}

## 3. Keras로 Stacked AutoEncoder 네트워크 만들어 보기 
Keras의 함수 API를 사용해 Stacked AutoEncoder를 만들어 보도록 하겠습니다.<br>
순차적 모델과 다르게 함수 API 모델은 각 은닉층의 출력을 변수로 할당하고, 다음 은닉층에 이전 은닉층의 출력 변수를 함수 파라미터로 입력하는 방식입니다.<br>
일반적인 경우에는 순차적 모델을 사용해도 상관없지만, 중간에 은닉층에서 출력을 추가적으로 얻거나 서로 다른 층에 연결하고자 할 때는 함수 API 모델을 사용해야 합니다.

In [ ]:
encoding_dim = 2
#입력 층은 784차원 벡터 입니다.
input_shape = (len(X[0]),)
#입력층
input_tensor = Input(shape = input_shape)
#인코더 부분 시작
x = layers.Dense(512,activation='relu')(input_tensor)
x = layers.Dense(256,activation='relu')(x)
x = layers.Dense(128,activation='relu')(x)
x = layers.Dense(32,activation='relu')(x)
x = layers.Dense(16,activation='relu')(x)
encoded = layers.Dense(encoding_dim, activation='relu')(x)
#디코더 부분
x = layers.Dense(16,activation='relu')(encoded)
x = layers.Dense(32,activation='relu')(x)
x = layers.Dense(128,activation='relu')(x)
x = layers.Dense(256,activation='relu')(x)
x = layers.Dense(512,activation='relu')(x)
#출력층
decoded = layers.Dense(784, activation='sigmoid')(x)

#오토인코더 모델
#순차적 모델에서는 models 패키지에 있는 Sequential Model 을 사용하였습니다. 
#함수형 API 모델에서는 입력 층과 출력 층을 models 패키지의 Model에 매개변수로 전달하여 네트워크를 생성할 수 있습니다.
autoencoder = models.Model(input_tensor, decoded)

#모델 생성 
#optimizer는 Adam 을 사용합니다.
autoencoder.compile(optimizer='adam', loss='mse')

함수형 API 모델에서도 동일하게 model.summary() 함수를 통해 네트워크의 구조를 파악할 수 있습니다.

In [ ]:
autoencoder.summary()

In [ ]:
# 입력과 동일한 출력을 만드는 모델이기 때문에 입력과 출력에 동일한 X(MNIST 이미지) 데이터를 사용해 학습합니다.
autoencoder.fit(X, X,
                epochs=20,
                batch_size=1024,
                shuffle=True,
               verbose=2)

이미지 하나를 복원해 확인해 봅시다.

In [ ]:
decoded_img = autoencoder.predict(X[4].reshape(1 ,784))

In [ ]:
img=plt.imshow(X[4].reshape((28,28)))

In [ ]:
img=plt.imshow(decoded_img.reshape((28,28)))

In [ ]:
label_dict[labels[4]]

이미지가 선명하지 않은 결과를 보였습니다.<br>
무엇이 문제였을까요? 784 차원의 데이터를 너무 작은 차원까지 축소 시켜서 그랬을 수 있습니다.<br>

## 4. 연습문제 : 차원을 조금만 축소시키는 AutoEncoder 만들기 
표시된 부분의 은닉층을 수정하여 이미지를 더 잘 복원하는 AutoEncoder를 만들어보세요.

In [ ]:
encoding_dim = 64
input_shape = (len(X[0]),)

input_tensor = Input(shape = input_shape)
'''
여기를 수정하여 인코더 부분의 은닉층을 자유롭게 만들어주세요.
인코더의 첫 번째 레이어는 다음 줄로 시작합니다.
x = layers.Dense(units ,activation='relu')(input_tensor)
'''

encoded = layers.Dense(encoding_dim, activation='relu')(x)
'''
여기를 수정하여 디코더 부분의 은닉층을 자유롭게 만들어 주세요.
디코더의 첫 번째 레이어는 다음 줄로 시작합니다.
x = layers.Dense(units ,activation='relu')(encoded)
'''
decoded = layers.Dense(784, activation='sigmoid')(encoded)

#전체 모델
autoencoder = models.Model(input_tensor, decoded)

#모델 생성 
#loss는 각 픽셀에 대해 예측하는 것이므로 binary_crossentropy를 사용합니다.
#optimizer는 Adam 을 사용합니다.
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.summary()

In [ ]:
# 입력과 동일한 출력을 만드는 모델이기 때문에 입력과 출력에 동일한 X(MNIST 이미지) 데이터를 사용해 학습합니다.
autoencoder.fit(X, X,
                epochs=20,
                batch_size=1024,
                shuffle=True,
               verbose=2)

In [ ]:
decoded_img = autoencoder.predict(X[1].reshape(1 ,784))

In [ ]:
img=plt.imshow(X[1].reshape((28,28)))

In [ ]:
img=plt.imshow(decoded_img.reshape((28,28)))

In [ ]:
label_dict[labels[1]]

## 5. Discussion
이전 모델보다 더욱 선명한 모델을 얻으셨나요? <br>
이제 불량 데이터인 임의 노이즈 데이터를 AutoEncoder로 복원시켜 보겠습니다.<br>
numpy의 random.normal 함수를 사용해 노이즈를 생성합니다. 파라미터로는 loc : 평균, scale : 표준편차, size : 차원 혹은 크기를 입력합니다.<br>

In [ ]:
notgood_data = np.random.normal(loc = 0, scale = 1, size = (28,28))

In [ ]:
img=plt.imshow(notgood_data)

In [ ]:
decoded_img = autoencoder.predict(notgood_data.reshape(1 ,784))
img=plt.imshow(decoded_img.reshape((28,28)))

임의 노이즈 데이터가 정상적으로 복원되지 않은 것을 확인할 수 있습니다.<br>
출력이 정상적으로 입력과 동일하게 복원된 것은 아니기 때문에 입력 데이터를 불량 데이터로 간주할 수 있습니다.<br>
이러한 AutoEncoder의 특성을 통해 이상치를 데이터에서 제거하고 클래스 불균형 문제를 완화할 수 있는 방법으로 사용할 수 있습니다.

### Reference
- AutoEncoder - https://blog.keras.io/building-autoencoders-in-keras.html